In [1]:
import numpy as np
import os
import shutil
import sys

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
datasets = '../datasets/'
glove_file_location = os.path.join(datasets, 'glove.6B/glove.6B.100d.txt')

In [4]:
%%time 
# Process the Glove vectors into a dictionary
embeddings_index = {}
max_database_size = 400000
with open(glove_file_location, encoding='utf-8') as f:
    for line in f:
        parsed_line = line.split()
        # The explicit string conversion below seems to fix a weird unicode runtime issue
        word = str(parsed_line[0]) 
        vector = np.array(parsed_line[1:], dtype='float32')
        embeddings_index[word] = vector
        if len(embeddings_index) >= max_database_size:
            break

print('Loaded {0} words from database.'.format(len(embeddings_index)))
    

Loaded 400000 words from database.
CPU times: user 13.4 s, sys: 430 ms, total: 13.9 s
Wall time: 14.1 s


### Tokenize raw imdb database

- Use 10000 words to tokenize
- Use 200 training samples only
- Use 10000 validation samples
- Use 100 words from each sample 

In [5]:
num_words = 10000
num_train = 200
num_validation = 10000
maxlen = 100

In [6]:
imdb_location = os.path.join(datasets, 'aclImdb')
train_location = os.path.join(imdb_location, 'train')

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [8]:
%%time

texts = []
labels = []
folders = ['neg', 'pos']

for folder in folders:
    complete_path = os.path.join(train_location, folder)
    for filename in os.listdir(complete_path):
        if filename[-4:] == '.txt':
            file = os.path.join(complete_path, filename)
            with open(file, encoding='utf-8') as f:
                review = f.read()
                texts.append(str(review))
                if folder == 'neg':
                    labels.append(0)
                else:
                    labels.append(1)
                
    print('Completed processing *{0}* folder. Total reviews: {1}'.format(folder, len(labels)))


Completed processing *neg* folder. Total reviews: 12500
Completed processing *pos* folder. Total reviews: 25000
CPU times: user 1.64 s, sys: 2.55 s, total: 4.18 s
Wall time: 5.81 s


In [9]:
tokenizer = Tokenizer( num_words )
tokenizer.fit_on_texts( texts )

In [10]:
tokenized_texts = tokenizer.texts_to_sequences(texts)

In [11]:
padded_sequences = pad_sequences(tokenized_texts, maxlen)

In [12]:
# Choose 200 training samples randomly (100 from positive and 100 from negative)
negative_indices = np.random.randint(0, 12500, size=100)
positive_indices = np.random.randint(12500, 25000, size=100)

data = padded_sequences[np.concatenate((negative_indices, positive_indices))]

labels_array = np.array(labels)
data_labels = labels_array[np.concatenate((negative_indices, positive_indices))]

In [13]:
p = np.random.permutation(len(data))
x_train = data[p]
y_train = data_labels[p]